In [1]:
import redis
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [2]:
redis_conn = redis.Redis(host="127.0.0.1", port=6379) 

In [4]:
sample_df = pd.read_csv("S4.csv", encoding='cp1252')
print(sample_df)

                     Zone      Country ISO3  Year  Population  Fortification  \
0                   WCANA  Afghanistan  AFG  1961   8954000.0              0   
1                   WCANA  Afghanistan  AFG  1961   8954000.0              0   
2                   WCANA  Afghanistan  AFG  1961   8954000.0              0   
3                   WCANA  Afghanistan  AFG  1961   8954000.0              0   
4                   WCANA  Afghanistan  AFG  1961   8954000.0              0   
...                   ...          ...  ...   ...         ...            ...   
111813  Subsaharan Africa     Zimbabwe  ZWE  2011  13359000.0              0   
111814  Subsaharan Africa     Zimbabwe  ZWE  2011  13359000.0              0   
111815  Subsaharan Africa     Zimbabwe  ZWE  2011  13359000.0              0   
111816  Subsaharan Africa     Zimbabwe  ZWE  2011  13359000.0              0   
111817  Subsaharan Africa     Zimbabwe  ZWE  2011  13359000.0              0   

        PCDEA       MDI   Tagname Micro

In [6]:
# Convert DataFrame to a PyArrow Table
table = pa.Table.from_pandas(sample_df)

# Create an in-memory buffer for serialization
sink = pa.BufferOutputStream()
writer = pa.ipc.new_stream(sink, table.schema)
writer.write_table(table)
writer.close()
# Store serialized data in Redis
redis_conn.set("example_key_for_object", sink.getvalue().to_pybytes())


True

In [7]:
# Retrieve the serialized data from Redis
retrieved_data = redis_conn.get("example_key_for_object")
print(retrieved_data)
# Read it back using Apache Arrow IPC
reader = pa.ipc.open_stream(pa.BufferReader(retrieved_data))
table_restored = reader.read_all()

# Convert back to Pandas DataFrame
df_restored = table_restored.to_pandas()
df_restored

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,Zone,Country,ISO3,Year,Population,Fortification,PCDEA,MDI,Tagname,Micronutrient,Units,Estimated Intake,Requirements,Prevalence of Inadequate Intake
0,WCANA,Afghanistan,AFG,1961,8954000.0,0,2997,0.773239,VITC,Vitamin C,mg,0.040922,0.049955,62.746266
1,WCANA,Afghanistan,AFG,1961,8954000.0,0,2997,0.773239,ZN,Zinc,mg,0.001677,0.001770,58.326559
2,WCANA,Afghanistan,AFG,1961,8954000.0,0,2997,0.773239,VITA_RAE,Vitamin A,mcg,0.000275,0.000463,84.225561
3,WCANA,Afghanistan,AFG,1961,8954000.0,0,2997,0.773239,CA,Calcium,mg,0.700043,0.783092,63.814563
4,WCANA,Afghanistan,AFG,1961,8954000.0,0,2997,0.773239,CU,Copper,mcg,0.001979,0.000569,0.090734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111813,Subsaharan Africa,Zimbabwe,ZWE,2011,13359000.0,0,2196,0.709258,VITA_RAE,Vitamin A,mcg,0.000209,0.000472,95.388542
111814,Subsaharan Africa,Zimbabwe,ZWE,2011,13359000.0,0,2196,0.709258,VITB12,Vitamin B12,mcg,0.000001,0.000002,33.580498
111815,Subsaharan Africa,Zimbabwe,ZWE,2011,13359000.0,0,2196,0.709258,VITB6A,Vitamin B6,mg,0.001168,0.000936,16.055914
111816,Subsaharan Africa,Zimbabwe,ZWE,2011,13359000.0,0,2196,0.709258,VITC,Vitamin C,mg,0.033915,0.051271,81.059120
